In [70]:
from torch import nn
import torch

device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

torch.device(device)

device(type='mps')

In [71]:
import pandas as pd

rcsb_data = pd.read_csv("data/rcsb/RCSB_PDB_Macromolecular_Structure_Dataset_with_Structural_Features.csv")

In [72]:
filtered_data = rcsb_data.loc[:,["Sequence", "Number of Residues", "Molecular Weight per Deposited Model", "Molecular Weight (Entity)", "R Free", "R Work", "Helix", "Sheet", "Coil"]]
filtered_data = filtered_data.dropna(subset=["Helix", "Sheet", "Coil"])

In [73]:
def amino_acid_tokenizer(amino_acid : str) -> torch.Tensor:
    amino_acid_tokens = {
        "A": 1,
        "C": 2,
        "D": 3,
        "E": 4,
        "F": 5,
        "G": 6,
        "H": 7,
        "I": 8,
        "K": 9,
        "L": 10,
        "M": 11,
        "N": 12,
        "P": 13,
        "Q": 14,
        "R": 15,
        "S": 16,
        "T": 17,
        "V": 18,
        "W": 19,
        "Y": 20,
        "X": 21,
        "U": 22,
        "O": 23
    }
    return torch.tensor([amino_acid_tokens[aa] for aa in amino_acid], dtype = torch.float32)

In [74]:
filtered_data["Sequence"] = filtered_data["Sequence"].apply(amino_acid_tokenizer)

# Padding sequences with zeros to make them all the same length
max_sequence_length = filtered_data["Sequence"].apply(len).max()
filtered_data["Sequence"] = filtered_data["Sequence"].apply(lambda x: torch.nn.functional.pad(x, (0, max_sequence_length - len(x)), "constant", 0))

In [75]:
scrambled_data = filtered_data.sample(frac=1)
scrambled_data = scrambled_data.reset_index(drop=True)

data_size = len(scrambled_data)
train_size = int(data_size * 0.8)
test_size = data_size - train_size
validation_size = int(train_size * 0.2)

train_data = scrambled_data.iloc[:train_size - validation_size]
validation_data = scrambled_data.iloc[train_size - validation_size:train_size]
test_data = scrambled_data.iloc[train_size:]

train_data = train_data.reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [76]:
train_input_df = train_data.loc[:,["Sequence", "Number of Residues", "Molecular Weight per Deposited Model", "Molecular Weight (Entity)", "R Free", "R Work"]]
train_output_df = train_data.loc[:,["Helix", "Sheet", "Coil"]]
validation_input_df = validation_data.loc[:,["Sequence", "Number of Residues", "Molecular Weight per Deposited Model", "Molecular Weight (Entity)", "R Free", "R Work"]]
validation_output_df = validation_data.loc[:,["Helix", "Sheet", "Coil"]]
test_input_df = test_data.loc[:,["Sequence", "Number of Residues", "Molecular Weight per Deposited Model", "Molecular Weight (Entity)", "R Free", "R Work"]]
test_output_df = test_data.loc[:,["Helix", "Sheet", "Coil"]]

In [77]:
def create_dataset(input_df, output_df):
    input_tensors = torch.tensor(input_df.drop(columns=["Sequence"]).values, dtype=torch.float32)
    input_tensor_sequences = torch.stack(tuple(input_df["Sequence"].values), 0)

    output_tensors = torch.tensor(output_df.values, dtype=torch.float32)

    return torch.utils.data.TensorDataset(input_tensors, input_tensor_sequences, output_tensors)

In [78]:
train_dataset = create_dataset(train_input_df, train_output_df)
validation_dataset = create_dataset(validation_input_df, validation_output_df)
test_dataset = create_dataset(test_input_df, test_output_df)

In [79]:
training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [80]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Optimizer
from torch.utils.data import DataLoader

In [81]:
def train(epoch : int,
          model : nn.Module,
          device: torch.device,
          train_dataloader : DataLoader,
          optimizer : Optimizer,
          loss_fn : nn.Module,
          tensorboard_writer : SummaryWriter = None) -> float:
    running_loss = 0.

    model.train(True)

    for idx, (input_tensors, input_tensor_sequences, output_tensors) in enumerate(train_dataloader):
        input_tensors = input_tensors.to(device)
        input_tensor_sequences = input_tensor_sequences.to(device)
        output_tensors = output_tensors.to(device)

        optimizer.zero_grad()

        output = model(input_tensors, input_tensor_sequences)
        loss = loss_fn(output, output_tensors)
  
        loss.backward()
        optimizer.step()
  
        running_loss += loss.item()

        if tensorboard_writer is not None:
            tensorboard_writer.add_scalar("Loss/train", loss.item(), epoch * len(train_dataloader) + idx)

    running_loss /= len(train_dataloader)

    if tensorboard_writer is not None:
        tensorboard_writer.add_scalar("Loss/train/epoch", running_loss, epoch)
    
    return running_loss

In [82]:
def validate(epoch : int, 
             model : nn.Module,
             device : torch.device,
             validation_dataloader : DataLoader,
             loss_fn : nn.Module,
             tensorboard_writer : SummaryWriter = None) -> float:
    running_loss = 0.

    model.eval()

    with torch.no_grad():
        for idx, (input_tensors, input_tensor_sequences, output_tensors) in enumerate(validation_dataloader):
            input_tensors = input_tensors.to(device)
            input_tensor_sequences = input_tensor_sequences.to(device)
            output_tensors = output_tensors.to(device)

            output = model(input_tensors, input_tensor_sequences)
            loss = loss_fn(output, output_tensors)

            running_loss += loss.item()

            if tensorboard_writer is not None:
                tensorboard_writer.add_scalar("Loss/validation", loss.item(), epoch * len(validation_dataloader) + idx)
    
    running_loss /= len(validation_dataloader)

    if tensorboard_writer is not None:
        tensorboard_writer.add_scalar("Loss/validation/epoch", running_loss, epoch)
    
    return running_loss

In [83]:
def test(model : nn.Module,
         device : torch.device,
         test_dataloader : DataLoader,
         loss_fn : nn.Module) -> float:
    running_loss = 0.

    model.eval()

    with torch.no_grad():
        for idx, (input_tensors, input_tensor_sequences, output_tensors) in enumerate(test_dataloader):
            input_tensors = input_tensors.to(device)
            input_tensor_sequences = input_tensor_sequences.to(device)
            output_tensors = output_tensors.to(device)

            output = model(input_tensors, input_tensor_sequences)
            loss = loss_fn(output, output_tensors)

            running_loss += loss.item()

    running_loss /= len(test_dataloader)

    return running_loss

In [84]:
from datetime import datetime

In [85]:
def train_model(model : nn.Module,
                device : torch.device,
                training_dataloader : DataLoader,
                validation_dataloader : DataLoader,
                test_dataloader : DataLoader,
                optimizer : Optimizer,
                loss_fn : nn.Module,
                epochs : int,
                model_name : str = "SecondCount") -> float:
    model.to(device)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    tensorboard_writer = SummaryWriter(f"runs/{model_name}_{timestamp}")

    best_validation_loss = float("inf")

    for epoch in range(epochs):
        train_loss = train(epoch, model, device, training_dataloader, optimizer, loss_fn, tensorboard_writer)
        validation_loss = validate(epoch, model, device, validation_dataloader, loss_fn, tensorboard_writer)

        print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {train_loss} - Validation Loss: {validation_loss}")

        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            torch.save(model.state_dict(), f"models/{model_name}_{timestamp}_{epoch}.pt")


    test_loss = test(model, device, test_dataloader, loss_fn)

    print(f"Test Loss: {test_loss}")

    tensorboard_writer.close()

    return test_loss

In [86]:
class SecondCountModel(nn.Module):
    def __init__(self, max_sequence_length : int, structural_features : int):
        super().__init__()

        self.max_sequence_length = max_sequence_length
        self.structural_features = structural_features

        # Encoder-Decoder LSTM Model

        self.encoder_lstm = nn.LSTM(input_size=max_sequence_length, hidden_size=128, num_layers=1, batch_first=True)
        self.decoder_lstm = nn.LSTM(input_size=max_sequence_length, hidden_size=128, num_layers=1, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Sequential(
            nn.Linear(128 + structural_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 3),
            nn.ReLU()
        )
    
    def forward(self, input_tensors, input_tensor_sequences):
        # Encoding & decoding the input sequences
        encoder_output, encoder_hidden = self.encoder_lstm(input_tensor_sequences)
        decoder_output, decoder_hidden = self.decoder_lstm(input_tensor_sequences, encoder_hidden)

        # Applying values from input_tensors and encoded_decoded to the fully connected layer
        concatenated = torch.cat((input_tensors, decoder_output), 1)
        return self.fc(concatenated)

In [87]:
model = SecondCountModel(max_sequence_length, train_dataset.tensors[0].shape[1])
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

In [88]:
train_model(model, device, training_dataloader, validation_dataloader, test_dataloader, optimizer, loss_fn, 128, "SecondCount")

Epoch 1/128 - Train Loss: 59402.222903588976 - Validation Loss: 53202.88383957435
Epoch 2/128 - Train Loss: 58865.83254298581 - Validation Loss: 52347.676892510775
Epoch 3/128 - Train Loss: 57377.99743279203 - Validation Loss: 50717.086442618536
Epoch 4/128 - Train Loss: 55205.195602483625 - Validation Loss: 47894.9819841056
Epoch 5/128 - Train Loss: 52619.71752183406 - Validation Loss: 45707.56623787715
Epoch 6/128 - Train Loss: 49609.37462046261 - Validation Loss: 42130.80334472656
Epoch 7/128 - Train Loss: 46508.13101716021 - Validation Loss: 39363.22994679418
Epoch 8/128 - Train Loss: 43460.52789813046 - Validation Loss: 36916.359240301725
Epoch 9/128 - Train Loss: 40564.13683389056 - Validation Loss: 33796.24590853987
Epoch 10/128 - Train Loss: 37983.433672642605 - Validation Loss: 32029.446987809806
Epoch 11/128 - Train Loss: 35749.94964519651 - Validation Loss: 29991.715769800645
Epoch 12/128 - Train Loss: 33864.02565502183 - Validation Loss: 27907.6593985722
Epoch 13/128 - Trai

26599.042073567707